In [4]:
# necessary libraries that I thought
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential

# Here reading the data set, I am reading both here only
train_data = pd.read_csv('Hindi_train.csv')
test_data = pd.read_csv('hindi_test.csv')


#some little preprocessing
#I creted the lebels "Text" and "Sentiment" as it was not in the data set
X_train = train_data['Sentence'].astype(str)
X_test = test_data['Sentence'].astype(str)

# As computers dont undertand words, converting it into numbers               #note I tried first with words:p
label_mapping = {'joy': 1, 'neutral': 0, 'surprise': 2, 'fear': 3, 'disgust' :4 , 'anger' :5, 'sadness' : 6}

y_train = train_data['Label'].map(label_mapping)
y_test = test_data['Label'].map(label_mapping)

# TF-IDF learnt in NLP, I liked this
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Simply taken Naive bayes fucntion from sklearn
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train_tfidf, y_train)

# Evaluation thing I checked
y_pred_nb = nb_classifier.predict(X_test_tfidf)
accuracy_nb = accuracy_score(y_test, y_pred_nb)
precision_nb = precision_score(y_test, y_pred_nb, average='weighted')
recall_nb = recall_score(y_test, y_pred_nb, average='weighted')
f1_nb = f1_score(y_test, y_pred_nb, average='weighted')

# LSTM model, took help
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)
X_train_pad = pad_sequences(X_train_seq, maxlen=200)
X_test_pad = pad_sequences(X_test_seq, maxlen=200)

model = Sequential([
    Embedding(input_dim=5000, output_dim=64, input_length=200),
    LSTM(64),
    Dense(7, activation='softmax')  # softmax for LSTM
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train_pad, y_train, epochs=1, batch_size=32, validation_data=(X_test_pad, y_test))

# Evaluatoin kinda same as above checked the difference
y_pred_dl_prob = model.predict(X_test_pad)
y_pred_dl = y_pred_dl_prob.argmax(axis=-1)
accuracy_dl = accuracy_score(y_test, y_pred_dl)
precision_dl = precision_score(y_test, y_pred_dl, average='weighted')
recall_dl = recall_score(y_test, y_pred_dl, average='weighted')
f1_dl = f1_score(y_test, y_pred_dl, average='weighted')


new_text = "I am sad"    #took a text, real life incident


#prediction work
new_text_tfidf = tfidf_vectorizer.transform([new_text])
predicted_sentiment_nb = nb_classifier.predict(new_text_tfidf)

print("Accuracy:", accuracy_dl)
print("Precision:", precision_dl)
print("Recall:", recall_dl)
print("F1 Score:", f1_dl)


new_text_seq = tokenizer.texts_to_sequences([new_text])
new_text_pad = pad_sequences(new_text_seq, maxlen=200)
predicted_sentiment_dl_prob = model.predict(new_text_pad)
predicted_sentiment_dl = predicted_sentiment_dl_prob.argmax(axis=-1)


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


17/17 [==============================] - 1s 25ms/step
Accuracy: 0.4619666048237477
Precision: 0.3074588783483662
Recall: 0.4619666048237477
F1 Score: 0.30453656262316514
1/1 [==============================] - 0s 31ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
